In [17]:
import numpy as np
import ee
import matplotlib.pyplot as plt
import tifffile as tiff
import zipfile
import os
import requests
from io import StringIO
import random

In [8]:
ee.Initialize()

In [13]:
def unzipURL(img_url, tmp_directory = None):
    """
    Downloads and unzips a file in a tmp directory.
    Hits the server once.

    img_url: A web address to a zip file.
    tmp_directory: Optional path to unzip the contents.

    Returns: Path to the directory containing the contents of the zip file.
    """

    #
    # Set up temp directory
    #
    if tmp_directory == None:
        tmp_directory="./tmp/"+str(random.randrange(99999999))
    zip_filename = os.path.join(tmp_directory, 'zipped_tile.zip')
    # make tmp directory
    if not os.path.exists(tmp_directory):
        os.makedirs(tmp_directory)
    # and delete everything in it:
    filelist = [ f for f in os.listdir(tmp_directory)]
    for f in filelist:
        os.remove(os.path.join(tmp_directory, f))

    #
    # Download file
    #
    try:
        r = requests.get(img_url)
        sc = r.status_code
    except requests.ConnectionError as e:
        raise e

    if r.status_code == 200:
        z = zipfile.ZipFile(StringIO.StringIO(r.content))
        z.extractall(tmp_directory)
        z.close()
        return tmp_directory
    elif r.status_code == 400:
        print ("Uh-oh. Status code 400 (Bad Request). Possible problems:")
        print ("- You requested a band that wasn't available in every image.")
        print ("- Some other unknown issue.")
        raise ServerError("Status code 400")
    elif r.status_code == 429:
        raise ServerError("Status code 429")
    else:
        raise ServerError("Uh-oh. Status code " + str(r.status_code))

    # Fall through on handled error/exception
    raise Exception("Unknown error in unzipURL().")


In [14]:
def tif_at_region(geCollection, resolution, bands, geo_bounds, verbose=False):
    DEFAULT_MAP_NAME = 'map_section'
    gee_img = geCollection.filterBounds(geo_bounds).select(bands).mosaic()
    bands_string = [{'id': b} for b in bands]
    path=gee_img.getDownloadUrl({
            'name': DEFAULT_MAP_NAME,
            'scale': resolution,
            'crs': 'EPSG:4326',
            'bands': bands_string,
            'region': geo_bounds.getInfo()['coordinates']
    });
    
    if verbose:
        print(path)
    
    tif_location = unzipURL(path)
    tif_filename_object = {}
    for band in bands:
        tif_filename = DEFAULT_MAP_NAME+'.'+band+'.tif'
        tif_path = os.path.join(tif_location, tif_filename)
        tif_filename_object[band] = tif_path
    return tif_filename_object

In [15]:
def img_at_region(geCollection, resolution, bands, geo_bounds, verbose=False):
    tif_band_dictionary = tif_at_region(geCollection, resolution, bands, geo_bounds, verbose)
    img_band_dictionary = {}
    for k,v in tif_band_dictionary.items():
        img_band_dictionary[k] = tiff.imread(v)
    return img_band_dictionary

In [18]:
nps_bounds = ee.Geometry.Rectangle([-121.880742, 36.600885, -121.866788, 36.594170])
tiles = img_at_region(ee.ImageCollection('USDA/NAIP/DOQQ'), 5, ['R'], nps_bounds)
plt.imshow(tiles['R'], cmap='gray')
plt.show()

AttributeError: type object '_io.StringIO' has no attribute 'StringIO'